<a href="https://colab.research.google.com/github/YKG-123/Fantasy_football_ML_Random_forest_Regressor/blob/main/QB_preseason_postable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive')
QB_2025=pd.read_csv('/content/drive/MyDrive/QB_2025.csv')
QB_2025.rename(columns={"adp":"adp_for_model"})
QB_2025.rename(columns={"adp_for_rank": "adp_true"}, inplace=True)
QB_2025=QB_2025.reset_index(drop=True)
QB_2025.drop(columns=["adp_true"], inplace=True)
QB_2025.rename(columns={"adp_true_y": "adp"}, inplace=True)

Mounted at /content/drive


In [ ]:
QB_2025.columns

Index(['player_name', 'season', 'week', 'offense_snaps', 'offense_pct',
       'team_offense_snaps_prev', 'birth_year', 'draft_year', 'draft_round',
       'draft_pick',
       ...
       'career_ypg', 'career_average_ppr_ppg', 'career_average_standard_ppg',
       'adp_true_x', 'player_name_merge', 'predicted_target_season', 'adp',
       'adp_rank', 'predicted_rank', 'rank_diff'],
      dtype='object', length=393)

In [ ]:

# --- 1. Ensure column uniqueness ---
QB_2025 = QB_2025.loc[:, ~QB_2025.columns.duplicated()]

# --- 2. Fill missing ADP with sentinel ---
QB_2025["adp"] = QB_2025["adp"].fillna(200)

# --- 3. Filter out undrafted ---
QB_2025 = QB_2025[QB_2025["adp"] != 200]  # keep only drafted players

# --- 4. Compute ranks ---
QB_2025["predicted_rank"] = QB_2025["predicted_target_season"].rank(ascending=False)
QB_2025["adp_rank"]       = QB_2025["adp"].rank(ascending=True)

# --- 5. ADP minus model prediction gap ---
QB_2025["adp-predictions"] = QB_2025["adp_rank"] - QB_2025["predicted_rank"]

# --- 6. Sort so biggest difference appears first ---
QB_2025 = QB_2025.sort_values(by="predicted_rank", ascending=True).reset_index(drop=True)
QB_2025["adp"] = QB_2025["adp"].round(2)
# --- 7. Preview top 75 WRs ---
QB_2025[["player_name", "adp", "predicted_rank", "adp_rank", "adp-predictions","predicted_target_season"]].head(100)



,player_name,adp,predicted_rank,adp_rank,adp-predictions,predicted_target_season
0,Lamar Jackson,21.4,1.0,1.0,0.0,24.746588
1,Josh Allen,25.1,2.0,2.0,0.0,24.741648
2,Jayden Daniels,32.6,3.0,4.0,1.0,24.729144
3,Kyler Murray,82.3,4.0,10.0,6.0,24.627884
4,Caleb Williams,103.0,5.0,13.0,8.0,24.316839
5,Bo Nix,70.3,6.0,8.0,2.0,24.307509
6,Patrick Mahomes,52.6,7.0,7.0,0.0,24.172064
7,Jalen Hurts,41.5,8.0,5.0,-3.0,24.129821
8,Baker Mayfield,52.0,9.0,6.0,-3.0,24.052384
9,Brock Purdy,76.3,10.0,9.0,-1.0,23.383125
